**Initialization and Setup**

In [10]:
import ee
from RadGEEToolbox import LandsatCollection
from RadGEEToolbox import Sentinel2Collection

In [11]:
# Attempt to initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully.")
except Exception as e:
    print("Initialization failed, attempting authentication...")
    try:
        ee.Authenticate()
        ee.Initialize()
        print("Authentication and initialization successful.")
    except Exception as auth_error:
        print("Authentication failed. Error details:", auth_error)


Earth Engine initialized successfully.


**Defining collections - required arguments are start and end dates, and either tile(s), boundary/geometry, or relative orbit(s) (orbits for Sentinel-2 only)**

Below is an example of multiple ways to define a Sentinel-2 collection

In [8]:
#General way to define a Sentinel-2 image collection using start date, end date, and a single MGRS tile
S2_col = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', tile='12TUL')

#General way to define a Sentinel-2 image collection using start date, end date, and multiple MGRS tiles
S2_col = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', tile=['12TUL', '12TUM', '12TUN'])

#Rather than using tiles, we can use a boundary (ee.Geometry) or if using Sentinel-2 we can use relative orbits - here we use a relative orbit number which provides a full swath of data
S2_col_orbit_filter = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', relative_orbit_number=127)

#Here is an example of using a boundary to filter the collection - first defining a boundary, in this case the county of Salt Lake City
counties = ee.FeatureCollection('TIGER/2018/Counties')
salt_lake_county = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Salt Lake'),
    ee.Filter.eq('STATEFP', '49')))
salt_lake_geometry = salt_lake_county.geometry()

S2_col_boundary_filter = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-15', boundary=salt_lake_geometry)

#You can filter for clouds by setting the cloud percentage threshold - here we set it to 15%
S2_col_low_clouds = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', relative_orbit_number=127, cloud_percentage_threshold=15)

#You can also filter for images with a lot of NoData - which happens more than you'd think - here we set the threshold to 15% as well
S2_col_no_blank_images = Sentinel2Collection(start_date='2023-06-01', end_date='2023-06-30', relative_orbit_number=127, nodata_threshold=15)


As GEE uses deferred execution, GEE doesn't immediately compute results when you define a collection. The processing is deferred until you explicityly ask for an output. This can be done by printing the dates of a defined collection, to verify the collection is defined correctly and is not an empty collection.

**Below is an example of how to quickly print the dates for any `RadGEEToolbox` collection object**

In [9]:
print(S2_col.dates)

['2023-06-01', '2023-06-01', '2023-06-02', '2023-06-02', '2023-06-04', '2023-06-04', '2023-06-04', '2023-06-06', '2023-06-06', '2023-06-07', '2023-06-07', '2023-06-09', '2023-06-09', '2023-06-09', '2023-06-11', '2023-06-11', '2023-06-12', '2023-06-12', '2023-06-12', '2023-06-14', '2023-06-14', '2023-06-14', '2023-06-16', '2023-06-16', '2023-06-17', '2023-06-17', '2023-06-19', '2023-06-19', '2023-06-19', '2023-06-21', '2023-06-21', '2023-06-22', '2023-06-22', '2023-06-22', '2023-06-24', '2023-06-24', '2023-06-24', '2023-06-26', '2023-06-26', '2023-06-27', '2023-06-27', '2023-06-29', '2023-06-29', '2023-06-29']


**Defining a LandsatCollection object is very similar to Sentinel-2, however, Landsat tiles use a different grid system (WRS-2) and thus the arguments for filtering by tile(s) are slightly different**

**Below are some examples of how to define and filter a Landsat collection**

In [5]:
# Similar examples for Landsat collections - showing how to filter using tiles or boundaries
col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', tile_row=32, tile_path=38)
tile_filtered_col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', tile_row=32, tile_path=38, cloud_percentage_threshold=50)
SLC_filtered_col = LandsatCollection(start_date='2023-06-01', end_date='2023-06-30', boundary=salt_lake_geometry, cloud_percentage_threshold=15)

**You may want to view the metadata of the defined collection to view band names and available properties - as shown below**

The `.collection` attribute converts the RadGEEToolbox collection object to an ee.ImageCollection to perform native GEE operations

In [6]:
# Showing how to print properties of an image collection 
# by turning the LandsatCollection object into an ee.ImageCollection object and using the .getInfo() parameter
print(SLC_filtered_col.collection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 167985, 0, -30, 4587315]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7891, 8001], 'crs':

**Data management is an important aspect of remote sensing workflows and RadGEEToolbox provides flexibility to convert image collections back and forth from RadGEEToolbox and GEE image collection objects, which allows inclusion of custom GEE API functions and image processing workflows while retaining the ability to use RadGEEToolbox functionality at any time**

If we print the variable of one of the collections we see it is a RadGEEToolbox object

In [ ]:
print(S2_col)

**Below is an example of how to convert back to a GEE collection then convert back to a RadGEEToolbox collection**

In [ ]:
# We can turn a Sentinel2Collection or LandsatCollection object into an Earth Engine image collection using the collection attribute
S2_gee_col = S2_col.collection
print('The collection is now a', type(S2_gee_col))

# Say you have an Earth Engine image collection object but you want to turn it into a Sentinel2Collection or LandsatCollection object, 
# just feed it in as a collection!
S2_col = Sentinel2Collection(collection=S2_gee_col)
print('The collection is back to a', type(S2_col))



The collection is now a <class 'ee.imagecollection.ImageCollection'>
The collection is back to a <class 'RadGEEToolbox.Sentinel2Collection.Sentinel2Collection'>
Readable list of image dates (client-side) ['2023-06-01', '2023-06-01', '2023-06-02', '2023-06-02', '2023-06-04', '2023-06-04', '2023-06-04', '2023-06-06', '2023-06-06', '2023-06-07', '2023-06-07', '2023-06-09', '2023-06-09', '2023-06-09', '2023-06-11', '2023-06-11', '2023-06-12', '2023-06-12', '2023-06-12', '2023-06-14', '2023-06-14', '2023-06-14', '2023-06-16', '2023-06-16', '2023-06-17', '2023-06-17', '2023-06-19', '2023-06-19', '2023-06-19', '2023-06-21', '2023-06-21', '2023-06-22', '2023-06-22', '2023-06-22', '2023-06-24', '2023-06-24', '2023-06-24', '2023-06-26', '2023-06-26', '2023-06-27', '2023-06-27', '2023-06-29', '2023-06-29', '2023-06-29']
Server side dates are of type: <class 'ee.ee_list.List'>


## Supplemental examples
### Examples of some useful functions are outlined in the cells below

**Attributes**

In [ ]:
# We can easily print the dates of all of the images in the collection using the dates attribute - this is a client-side operation
S2_dates = S2_col.dates
print('Readable list of image dates (client-side)', S2_dates)

# Alternatively, we can make a list of server-side dates for iteration, if needed
S2_dates_server_side = S2_col.dates_list
print('Server side dates are of type:', type(S2_dates_server_side))

# You can easily mask out clouds or water in the image collections
S2_masked_clouds = S2_col.masked_clouds_collection
S2_masked_to_water = S2_col.masked_to_water_collection

**Utilizing methods for general data management**
- Masking to polygon
- Mosacing images with same date
- Masking water out of images automatically
- Masking images to water automatically

In [9]:
# Here are some examples of utilizing method functions and attributes of the Sentinel2Collection and LandsatCollection classes

# Mask entire collection based on geometry
masked_S2_col = S2_col_orbit_filter.mask_to_polygon(salt_lake_geometry)

# Mosaic images in collection that share an image date
mosaiced_S2_col = S2_col_boundary_filter.MosaicByDate

# Mask water pixels from each single image using quality bands
water_masked_S2_col = S2_col_boundary_filter.masked_water_collection

# Mask water pixels from each single image using NDWI - where values less than the specified threshold are masked in each image
water_masked_S2_col = S2_col_boundary_filter.masked_water_collection_NDWI(threshold=0)

**Example chaining of methods - where we first mosaic the collection, mask the collection to water pixels, then calculate relative turbidity for each image**

In [10]:
# Example chaining of methods - where we first mosaic the collection, mask the collection to water pixels, then calculate relative turbidity for each image

turbidity_chain_example = S2_col.MosaicByDate.masked_to_water_collection.turbidity
print(turbidity_chain_example.dates)

['2023-06-01', '2023-06-02', '2023-06-04', '2023-06-06', '2023-06-07', '2023-06-09', '2023-06-11', '2023-06-12', '2023-06-14', '2023-06-16', '2023-06-17', '2023-06-19', '2023-06-21', '2023-06-22', '2023-06-24', '2023-06-26', '2023-06-27', '2023-06-29']


**Arguably, one of the most useful method functions is image_grab() or image_pick() which allow you to iteratively select images from an image collection**

In [11]:
# Select image from collection based on index
image_from_S2_collection = mosaiced_S2_col.image_grab(-1)

# Select image from collection based on date
image_from_S2_collection = mosaiced_S2_col.image_pick(mosaiced_S2_col.dates[-1])

print(image_from_S2_collection.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'int'

**Using static functions**

In [12]:
# adding image date properties to images (very general example)
S2_col_date_props = S2_col.collection.map(Sentinel2Collection.image_dater)

# masking water from images
S2_water_masked_col = S2_col.collection.map(Sentinel2Collection.MaskWaterS2)

# calculating the surface area of pixels of interest as square meters(water pixels from NDWI for example)
water_area = Sentinel2Collection.PixelAreaSum(image=mosaiced_S2_col.ndwi.image_grab(-1), band_name='ndwi', geometry=salt_lake_geometry, threshold=0, scale=10)

print('Square meters of water in image:', water_area.getInfo().get('properties')['ndwi'])

Square meters of water in image: 1647329756.0751777


In [17]:
# Showing how to make an image collection with pixel area calculated for all images in the collection (using ndwi images as example), and how to assess 
# the area calculations using aggregate_array()

area_col = mosaiced_S2_col.ndwi.PixelAreaSumCollection(band_name='ndwi', geometry=salt_lake_geometry, threshold=0, scale=50)
print('Square meters of water in images are:', area_col.aggregate_array('ndwi').getInfo())
print('Dates of images:', mosaiced_S2_col.dates)

Square meters of water in images are: [1069594768.6520188, 251369607.87758926, 247235936.65393806, 496807659.3774899, 192290154.215422, 1646335856.4432714]
Dates of images: ['2023-06-01', '2023-06-04', '2023-06-06', '2023-06-09', '2023-06-11', '2023-06-14']


### *Please refer to the [RadGEEToolbox documentation](https://radgeetoolbox.readthedocs.io/en/latest/) for more information and a comprehensive list of available functionality*